### <font color='#4285f4'>Overview</font>

This process generates synthetic data for marketing campaigns, including campaign details, performance metrics, content, and recommendations. It simulates historical campaigns with defined budgets, target audiences, expected outcomes, and performance data, providing a comprehensive dataset for analysis and testing.

Process Flow:

1. Creates the BigQuery table: chocolate_ai.campaign
2. Creates the BigQuery table: chocolate_ai.campaign_performance
3. Generates Campaign Recommendations: name, description, target audience, expected outcome and explanation
4. Get the Ranked Products Monthly
5. Create Campaign Budgets
6. Generate Historical Campaigns
7. Set the dates
8. Simulate Campaign Performance


Cost:
* Low: Gemini, BigQuery
* Medium: Remember to stop your Colab Enterprise Notebook Runtime

Author: 
* Paul Ramsey

### <font color='#4285f4'>License</font>

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

### <font color='#4285f4'> Pip Installs</font>

In [ ]:
# PIP Installs
import sys

# https://PLACEHOLDER.com/index.html

# For better performance and production, deploy to Vertex AI endpoint with GPU
# This takes about 5 minutes to install and you will need to reset your runtime
#!{sys.executable} -m pip install timesfm

### <font color='#4285f4'>Initalize</font>

In [ ]:
from PIL import Image
from IPython.display import HTML
from IPython.display import Audio
from functools import reduce
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
from datetime import datetime, timedelta
import sys
import numpy as np
import logging

from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
# Update these variables to match your environment
location="us-central1" # Your region
bigquery_location = "${bigquery_location}" # Must be "us" or "eu"

### Do not change the values in this cell below this line ###
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")
print(f"location = {location}")
print(f"bigquery_location = {bigquery_location}")

### <font color='#4285f4'>Helper Methods</font>

#### RetryCondition(error)

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### restAPIHelper()
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### GeminiLLM (Pro 1.0 , Pro 1.5)

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### RunQuery(sql)

In [ ]:
def RunQuery(sql, job_config = None):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    if job_config == None:
      job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

#### PrettyPrintJson(json_string)

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

### <font color='#4285f4'>Create Tables</font>

In [ ]:
# Create the campaign table

%%bigquery

CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_chocoate_ai_dataset}.campaign` (
    campaign_id STRING NOT NULL OPTIONS(description="Unique identifier for each campaign (e.g., 'SummerSale2024', 'HolidayPromoDec')"),
    menu_id INT64 OPTIONS(description="Id of the menu item being promoted in the campaign"),
    campaign_name STRING OPTIONS(description="A descriptive name for the campaign."),
    campaign_description STRING OPTIONS(description="A detailed description of the campaign."),
    campaign_goal STRING OPTIONS(description="The objective of the campaign (e.g., 'Increase brand awareness', 'Drive website traffic', 'Generate leads')"),
    target_audience STRING OPTIONS(description="The intended audience for the campaign (e.g., 'Women aged 25-35', 'Small business owners in Texas')"),
    marketing_channels STRING OPTIONS(description="The marketing channels used in the campaign (e.g., 'Social Media', 'Email', 'SEO', 'Paid Ads')"),
    budget FLOAT64 OPTIONS(description="The overall budget allocated for the campaign."),
    explanation STRING OPTIONS(description="The model reasoning for creating the campaign."),
    campaign_outcomes STRING OPTIONS(description="Expected outcomes of the  campaign."),
    campaign_start_date DATE OPTIONS(description="The date the campaign started."),
    campaign_end_date DATE OPTIONS(description="The date the campaign ended."),
    campaign_created_date DATETIME OPTIONS(description="The date the campaign was created.")
) CLUSTER BY campaign_id;

In [ ]:
# Create the campaign_performance table

%%bigquery

CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_chocoate_ai_dataset}.campaign_performance` (
    campaign_id STRING NOT NULL OPTIONS(description="Foreign key referencing the campaign table. Unique identifier for the campaign."),
    impressions INT64 OPTIONS(description="Number of times the campaign was displayed."),
    reach INT64 OPTIONS(description="Number of unique individuals exposed to the campaign."),
    website_traffic INT64 OPTIONS(description="Visits to your website attributed to the campaign."),
    leads_generated INT64 OPTIONS(description="Number of new leads captured (e.g., email sign-ups, contact form submissions)."),
    conversions INT64 OPTIONS(description="Number of desired actions taken (e.g., purchases, downloads)."),
    cost_per_click FLOAT64 OPTIONS(description="Average cost per click on your ads (if applicable)."),
    cost_per_acquisition FLOAT64 OPTIONS(description="Average cost to acquire a new customer."),
    return_on_investment FLOAT64 OPTIONS(description="Return on Investment calculated as (Revenue - Cost) / Cost.")
) CLUSTER BY campaign_id;

### <font color='#4285f4'>Define Campaign Functions</font>

#### GenerateCampaignRecommendations()

In [ ]:
# Use tenacity to retry in case of resource exhausted errors (429)
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GenerateCampaignRecommendations(campaign_goal, product_name, product_description, budget):

  # For each row in customer_marketing_profile, send this prompt:
  prompt = f"""
  You are an expert Marketing Analyst, and you are creating a new marketing campaign for a
  French chocolatier and bakery called 'Chocolate AI'.

  Your task is to recommend a campaign for the product that will achieve the campaign goals within budget.

  You should include the following information in your response:
    - A creative campaign name.
    - A detailed description of the campaign.
    - The ideal target audience.
    - Ideal marketing channels. This should be optimized for the target audience.
    - Expected campaign outcomes. This should map directly to the campaign goals.
    - An explanation/justification for the reasoning behind choosing this campaign.

  Your total budget for the campaign is: {budget}

  Here is the goal of the campaign:
  {campaign_goal}

  Here is the product you are promoting:
  {product_name}: {product_description}

  Additional instructions:
  - Results can be creative, but they must be plausible.

  Now generate the campaign recommendation. Think step by step and explain your reasoning.
  """

  # Ref: https://cloud.google.com/vertex-ai/docs/reference/rest/v1/Schema
  response_schema = {}

  response_schema = {
    "type": "object",
    "required": ["campaign_name", "campaign_description", "target_audience", "marketing_channels", "campaign_outcomes", "explanation"],
    "properties": {
      "campaign_name": {
        "type": "string",
        "description": "A descriptive and creative name for the campaign."
      },
      "campaign_description": {
        "type": "string",
        "description": "A detailed description of the campaign."
      },
      "target_audience": {
        "type": "string",
        "description": "The intended audience of the campaign."
      },
      "marketing_channels": {
        "type": "string",
        "description": "The recommended marketing channels to be used in the campaign."
      },
      "campaign_outcomes": {
        "type": "string",
        "description": "The expected outcomes of the campaign."
      },
      "explanation": {
        "type": "string",
        "description": "An explanation for the reasoning behind choose this campaign."
      }
    }
  }

  result = GeminiLLM(prompt, response_schema=response_schema)
  return result


#### GetRankedProductsMonthly()

In [ ]:
# Get lowest-performing products based on month over month sales data
def GetRankedProductsMonthly(count, top_bottom = 'top'):
  sort_order = 'DESC' if top_bottom == 'top' else 'ASC'

  sql=f"""WITH current_month_sales AS (
    SELECT
        m.menu_name,
        m.menu_id,
        m.menu_description,
        SUM(oi.item_total) AS cumulative_sales
      FROM
        `${project_id}.${bigquery_chocoate_ai_dataset}.order_item` AS oi
        INNER JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.order` AS o ON o.order_id = oi.order_id
        INNER JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.menu` AS m ON m.menu_id = oi.menu_id
      WHERE o.order_datetime >= TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY))
      GROUP BY menu_name, m.menu_id, m.menu_description
      ORDER BY cumulative_sales
  ), prior_month_sales AS (
    SELECT
        m.menu_name,
        m.menu_id,
        m.menu_description,
        SUM(oi.item_total) AS cumulative_sales
      FROM
        `${project_id}.${bigquery_chocoate_ai_dataset}.order_item` AS oi
        INNER JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.order` AS o ON o.order_id = oi.order_id
        INNER JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.menu` AS m ON m.menu_id = oi.menu_id
      WHERE o.order_datetime BETWEEN TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)) AND TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY))
      GROUP BY menu_name, m.menu_id, m.menu_description
      ORDER BY cumulative_sales
  ) SELECT current_month_sales.menu_id,
    current_month_sales.menu_name,
    current_month_sales.menu_description,
    FORMAT('%s €', CAST(ROUND(current_month_sales.cumulative_sales, 0) as STRING)) AS current_month_cumulative_sales,
    FORMAT('%s €', CAST(ROUND(prior_month_sales.cumulative_sales, 0) as STRING)) AS prior_month_cumulative_sales,
    FORMAT('%s €', CAST(ROUND(current_month_sales.cumulative_sales - prior_month_sales.cumulative_sales, 0) as STRING))  AS change,
    ROUND((current_month_sales.cumulative_sales - prior_month_sales.cumulative_sales) / prior_month_sales.cumulative_sales * 100,0) AS change_percent
  FROM current_month_sales
  JOIN prior_month_sales ON current_month_sales.menu_name = prior_month_sales.menu_name
  ORDER BY change_percent {sort_order}
  LIMIT {count};
  """

  return RunQuery(sql)


#### GenerateCampaignPerformance(budget)

In [ ]:
def GenerateCampaignPerformance(budget):
  """
  Generates randomized campaign performance data, which is influence by budget.

  Args:
    campaign_id: The ID of the campaign.
    budget: The budget for the campaign.

  Returns:
    A dictionary containing the performance data.

  Example:
    budget = 5000
    performance_data = GenerateCampaignPerformance(budget)
    print(performance_data)
  """

  # Generate random scale factors within a range for non-linear relationship
  impressions_factor = random.uniform(0.6, 0.8)
  reach_factor = random.uniform(0.5, 0.7)
  traffic_factor = random.uniform(0.4, 0.6)
  leads_factor = random.uniform(0.3, 0.5)
  conversions_factor = random.uniform(0.2, 0.4)

  # Generate data points with randomness and scaling
  impressions = int(random.uniform(0.8, 1.2) * (budget ** impressions_factor) * 100)
  reach = int(random.uniform(0.7, 1.1) * (budget ** reach_factor) * 100)
  website_traffic = int(random.uniform(0.6, 1.0) * (budget ** traffic_factor) * 10)
  leads_generated = int(random.uniform(0.5, 0.9) * (budget ** leads_factor) * 5)
  conversions = int(random.uniform(0.4, 0.8) * (budget ** conversions_factor) * 2)


  # Cost per click and acquisition (with some randomness)
  cost_per_click = round(random.uniform(0.2, 1.5), 2)
  cost_per_acquisition = round(random.uniform(5, 50) * (15000 / budget), 2)

  # ROI (mostly positive, but some negative)
  roi_sign = random.choices([-1, 1], weights=[0.2, 0.8])[0]  # 20% chance of negative ROI
  return_on_investment = round(roi_sign * random.uniform(0.5, 10.0), 2)

  return {
      "impressions": impressions,
      "reach": reach,
      "website_traffic": website_traffic,
      "leads_generated": leads_generated,
      "conversions": conversions,
      "cost_per_click": cost_per_click,
      "cost_per_acquisition": cost_per_acquisition,
      "return_on_investment": return_on_investment
  }

### <font color='#4285f4'>Generate Historical Campaigns</font>

#### Build Campaigns

In [ ]:
# Get 52 products to promote for 1-year history
products_to_promote = GetRankedProductsMonthly(52, 'bottom')

# Define goal array
campaign_goal_array = [
    "Increase sales of the product by 15%.",
    "Drive 10,000 unique visitors to the product landing page.",
    "Generate 500 qualified leads for the sales team.",
    "Boost brand awareness by 20% among the target audience.",
    "Achieve a 5% click-through rate on online ads.",
    "Increase social media engagement by 30%.",
    "Improve customer satisfaction with the product by 10%.",
    "Gather 200 customer testimonials for marketing materials.",
    "Increase email sign-up rate by 5%.",
    "Drive 1,000 app downloads.",
    "Increase website traffic from organic search by 25%.",
    "Reduce customer churn rate by 10%.",
    "Increase average order value by 5%.",
    "Improve customer lifetime value by 15%.",
    "Generate 100 product reviews with an average rating of 4.5 stars.",
    "Increase the number of repeat purchases by 20%.",
    "Increase conversion rate from leads to customers by 10%.",
    "Expand into a new target market segment.",
    "Launch a successful new product line.",
    "Increase customer loyalty and brand advocacy.",
    "Improve brand reputation and sentiment.",
    "Increase share of voice in the industry.",
    "Become a thought leader in the industry.",
    "Build a strong online community around the brand.",
    "Increase customer engagement with email marketing.",
    "Improve the effectiveness of social media advertising.",
    "Optimize the customer journey for better conversion rates.",
    "Increase customer retention through personalized marketing.",
    "Drive sales through influencer marketing campaigns.",
    "Increase brand visibility through public relations efforts.",
    "Generate leads through content marketing initiatives.",
    "Improve customer experience across all touchpoints.",
    "Increase customer satisfaction with customer service.",
    "Reduce customer acquisition costs by 10%.",
    "Increase return on investment (ROI) for marketing campaigns.",
    "Improve the efficiency of marketing operations.",
    "Increase the speed and agility of marketing campaigns.",
    "Leverage data and analytics to improve marketing decisions.",
    "Adopt new marketing technologies to enhance performance.",
    "Increase the effectiveness of marketing automation.",
    "Improve lead nurturing processes to increase conversion rates.",
    "Increase sales through affiliate marketing programs.",
    "Drive traffic and sales through online contests and giveaways.",
    "Generate leads through webinars and online events.",
    "Increase brand awareness through sponsorships and partnerships.",
    "Improve customer communication through personalized messaging.",
    "Increase customer engagement through interactive content.",
    "Drive sales through retargeting campaigns.",
    "Increase customer lifetime value through loyalty programs.",
    "Build a strong brand identity and messaging."
]

# Define budget array
budget_array = np.linspace(500, 15000, num=52, dtype=int).tolist()

# Generate campaign recommendations for low-performing products
for row in products_to_promote.itertuples():
  campaign_goal = random.choice(campaign_goal_array)
  budget = random.choice(budget_array)


  product_name	= row.menu_name
  product_description = row.menu_description
  menu_id = row.menu_id

  result = GenerateCampaignRecommendations(campaign_goal, product_name, product_description, budget)
  print(result)

  # Save to database
  # Using json.dumps() to avoid unexpected quoting errors on insert
  json_result = json.loads(result)
  campaign_name = json.dumps(json_result["campaign_name"])
  campaign_description = json.dumps(json_result["campaign_description"])
  campaign_outcomes = json.dumps(json_result["campaign_outcomes"])
  target_audience = json.dumps(json_result["target_audience"])
  marketing_channels = json.dumps(json_result["marketing_channels"])
  explanation = json.dumps(json_result["explanation"])
  campaign_goal = json.dumps(campaign_goal)
  budget = budget

  sql=f"""
  INSERT INTO `${project_id}.${bigquery_chocoate_ai_dataset}.campaign`
    (campaign_id, campaign_name, campaign_description, campaign_goal, target_audience, marketing_channels, budget, explanation, campaign_outcomes, menu_id)
  VALUES
    (GENERATE_UUID(), {campaign_name}, {campaign_description}, {campaign_goal}, {target_audience}, {marketing_channels}, {budget}, {explanation}, {campaign_outcomes}, {menu_id});
  """

  RunQuery(sql)


#### Set Campaign Dates

In [ ]:
# Set historical campaign dates
sql="""SELECT campaign_id, campaign_name, campaign_start_date, campaign_end_date
FROM  `${project_id}.${bigquery_chocoate_ai_dataset}.campaign`;
"""

campaign_table = RunQuery(sql)

# Set seed date
seed_start_date = '2024-09-09'
seed_end_date = '2024-09-15'

# Convert the string dates to datetime objects
new_start_date_dt = datetime.strptime(seed_start_date, '%Y-%m-%d')
new_end_date_dt = datetime.strptime(seed_end_date, '%Y-%m-%d')

for row in campaign_table.itertuples():
  campaign_id = row.campaign_id

  # Update database
  sql=f"""UPDATE `${project_id}.${bigquery_chocoate_ai_dataset}.campaign`
    SET campaign_start_date = '{new_start_date_dt.strftime('%Y-%m-%d')}',
        campaign_end_date = '{new_end_date_dt.strftime('%Y-%m-%d')}',
        campaign_created_date = '{new_start_date_dt.strftime('%Y-%m-%d')}'
    WHERE campaign_id = '{campaign_id}';
  """

  print(f"Processing row {row.Index} for campaign {campaign_id}")
  result = RunQuery(sql)

  # Subtract one week (7 days) for next iteration
  new_start_date_dt = new_start_date_dt - timedelta(weeks=1)
  new_end_date_dt = new_end_date_dt - timedelta(weeks=1)


#### Simulate Campaign Performance

In [ ]:
# Generate historical campaign performance
sql="""SELECT campaign_id, budget
FROM `${project_id}.${bigquery_chocoate_ai_dataset}.campaign`;
"""

campaign_table = RunQuery(sql)

values = ""
gen_campaign_performance = [(row.campaign_id, *[v for v in GenerateCampaignPerformance(row.budget).values()]) for row in campaign_table.itertuples()]

for row in gen_campaign_performance:
  values = values + f"{row},\n"

values = values[:-2]

sql=f"""INSERT INTO `${project_id}.${bigquery_chocoate_ai_dataset}.campaign_performance` (
    campaign_id, impressions, reach, website_traffic, leads_generated, conversions, cost_per_click, cost_per_acquisition, return_on_investment)
    VALUES {values}
"""

result = RunQuery(sql)
print(result)


### <font color='#4285f4'>Clean Up</font>

In [ ]:
# Placeholder

### <font color='#4285f4'>Reference Links</font> 


- [Google.com](https://www.google.com)